In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import mean_squared_error


In [ ]:
data_dir="/kaggle/input/tesla-stock-data-updated-till-28jun2021/TSLA.csv"

df=pd.read_csv(data_dir, index_col='Date')
df.head(10)

In [ ]:
df.info()

In [ ]:
dataset=df[['Close']]

dataset

In [ ]:
dataset.shape

In [ ]:
dataset.plot(figsize=(12,6))

In [ ]:
train_size=int(0.8*len(dataset))
test_size=len(dataset)- train_size
print(f'Train Size: {train_size}')
print(f"Test Size: {test_size}")

In [ ]:
data=dataset.values

train_df=data[:train_size,:]

test_df=data[train_size-60:,:]

In [ ]:
X_train = []
y_train = []

for i in range(60, len(train_df)):
    X_train.append(train_df[i-60:i, :])
    y_train.append(train_df[i, :])

In [ ]:
X_train=np.array(X_train)
y_train=np.array(y_train)

In [ ]:
model = Sequential([
    LSTM(64,return_sequences=True, activation='tanh',input_shape=(X_train.shape[1:])),
    LSTM(32,return_sequences=False,activation='tanh'),
    Dense(64,activation='relu'),

    Dense(1,activation='linear')
])

model.compile(optimizer='adam',loss='mse')

In [ ]:
model.fit(X_train,y_train,epochs=200)

In [ ]:
# Creating a testing set with 60 time-steps and 1 output
x_test = []
y_test = []

for i in range(60, len(test_df)):
    x_test.append(test_df[i-60:i, 0])
    y_test.append(test_df[i, 0])

x_test, y_test = np.array(x_test), np.array(y_test)

In [ ]:
predictions=model.predict(x_test)
predictions

In [ ]:
train = dataset.iloc[:train_size , 0:1]
test = dataset.iloc[train_size: , 0:1]
test['Predictions'] = predictions

plt.figure(figsize= (16, 6))
plt.title('Tesla Close Stock Price Prediction', fontsize= 18)
plt.xlabel('Date', fontsize= 18)
plt.ylabel('Close Price', fontsize= 18)
plt.plot(train['Close'], linewidth= 3)
plt.plot(test['Close'], linewidth= 3)
plt.plot(test["Predictions"], linewidth= 3)
plt.legend(['Train', 'Test', 'Predictions'])